In [1]:
import random, numpy as np, pandas as pd
from matplotlib import pyplot as plt
alphap = 3
probp = 0.75
expon = 2 * alphap - 1
slope = 0.78835

In [2]:
def betadist(alpha):
    """gives a random number from beta distribution"""
    return random.betavariate(alpha,alpha)

In [3]:
def decision(probability):
    """
    decides with a given probability whether to keep the right part
    """
    if float(probability) > random.random():
        return True
    else: 
        return False

In [4]:
def splitting(segment):
    """
    splits a given segment. left and right are endpoints of the segment
    returns : 
        xL -> length of the left segment
        xR -> length of the right segment
        flag -> keeping the right segment
        xLp, xRp -> probability(unnormalized) for being selected
        change -> change of normalization const
    """
    xL = segment * betadist(alphap)
    xR = segment - xL
    flag = decision(probp)
    xLp = xL**expon
    xRp = xR**expon
    change = xLp + xRp - segment**expon
    return xL, xR, flag, xLp, xRp, change

In [5]:
def pickindex(frag_prob, frag_prob_sum):
    """
    picks up a segment to be subsequently split
    """
    r = random.uniform(0, frag_prob_sum)
    sum_ = 0
    for index in range(len(frag_prob)):
        sum_ += frag_prob[index]
        if sum_ < r:
            continue
        else:
            return index

In [6]:
def number_length(segment_lengths,flags):
    
    N = 0
    M = 0

    for i in range(len(flags)):
        if flags[i]:
            N += 1
            M += segment_lengths[i]
            pass
        pass
    return N, M

In [7]:
fractal_dim = slope

def fractal_length(segment_lengths,flags):

    M_frac = 0

    for i in range(len(flags)):
        if flags[i]:
            M_frac += segment_lengths[i]**fractal_dim
            pass
        pass
    return M_frac

In [8]:
def realization_value(total_iteration):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    frag_prob_sum = 1.0 # normalization const
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        index = pickindex(frag_prob, frag_prob_sum)
        
        if index != None and flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            frag_prob_sum += change
            pass
        
    M_frac = fractal_length(lengths,flags)
    
    return M_frac

In [9]:
def realization_value(total_iteration, min_iteration, iteration_step, data_point = False):
    
    lengths = [1.]
    flags = [True]
    frag_prob = [1.] # raw probability, not normalized
    frag_prob_sum = 1.0 # normalization const

    iteration_list = list(range(min_iteration, total_iteration + 1, iteration_step))
    
    M_realization = []
    
    for i in range(total_iteration + 1):
        
        index = pickindex(frag_prob, frag_prob_sum)
        
        if flags[index] == True:

            xL, xR, flag, xLp, xRp, change = splitting(lengths[index])
            
            lengths[index] = xL
            lengths.append(xR)
            flags.append(flag)
            frag_prob[index] = xLp 
            frag_prob.append(xRp)
            frag_prob_sum += change
            pass
        
        if i+1 in iteration_list:
            M_frac = fractal_length(lengths,flags)
            M_realization.append(M_frac)
        pass 
      
    if data_point:
        print(np.array(M_realization))
    
    else:
        M_frac = fractal_length(lengths,flags)
        return M_frac

In [10]:
def ensemble_average(total_iteration, ensemble_size):

    M_ensemble = 0
    
    for i in range(ensemble_size):
        M_list = realization_value(total_iteration, total_iteration, total_iteration)
        M_ensemble += M_list
        pass
    
    M_average = M_ensemble/ensemble_size
    
    print(M_average)

In [11]:
for i in range(5):
    realization_value(10_000_000, 1_000_000, 1_000_000, data_point = True)

[0.87094239 0.88303742 0.88543081 0.87667259 0.8868398  0.87662621
 0.87911391 0.88306665 0.87425139 0.87581064]
[1.02210947 1.01787397 1.03287968 1.03963492 1.05896808 1.05896808
 1.06420441 1.06933719 1.07803208 1.07063435]
[0.35031476 0.35031476 0.35031476 0.35031476 0.35031476 0.35719396
 0.35719396 0.35719396 0.35719396 0.36558958]
[0.03112768 0.03112768 0.03112768 0.03112768 0.03112768 0.03112768
 0.03112768 0.03112768 0.03112768 0.03112768]
[1.72832261 1.72247482 1.73508698 1.7246876  1.73608568 1.73441246
 1.74265597 1.73782018 1.74299582 1.7484177 ]


In [ ]:
for i in range(10):
    a_ = 1_000*(i+1)
    ensemble_average(a_, 100_000)

0.9992680491893363
